## Simulate running on multiple GPUs

In [1]:
from dask.distributed import Client

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import os
os.chdir('/home/ubuntu/adadamp-experiments/')

import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import csv
import importlib
import pandas as pd
from time import time
from copy import copy

In [4]:
# import simulator
os.chdir('/home/ubuntu/adadamp-experiments/')
classifier = importlib.import_module("exp-dask.classifier")
from classifier import DaskClassifierSimulator

In [5]:
# training client
from dask.distributed import Client

def _prep():
    from distributed.protocol import torch

client = Client(processes=False)
client.run(_prep)
client

Client Scheduler: inproc://172.31.40.124/2912/1 Dashboard: http://172.31.40.124:8787/status,Cluster Workers: 1 Cores: 4 Memory: 16.48 GB


In [6]:
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])
train_set = torchvision.datasets.CIFAR10(root='./exp-dask/data', train=True, download=True, transform=transform_train)
test_set = torchvision.datasets.CIFAR10(root='./exp-dask/data', train=False, download=True, transform=transform_test)
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [7]:
from model import Wide_ResNet

client.upload_file("./exp-dask/model.py")

In [8]:
def write_stats(hist, directory, filename):
    toCSV = hist
    with open('./exp-dask/{}/{}'.format(directory, filename), 'w', encoding='utf8', newline='') as output_file:
        fc = csv.DictWriter(output_file, fieldnames=toCSV[0].keys())
        fc.writeheader()
        fc.writerows(toCSV)

In [9]:
def train(model, train_set, test_set, stats_df, n_epochs=200, log_interval=1, stats_dir=''):
    """
    Train based on expiriment params
    
    Parameters:
    epoch_sched: update lr and bs at epochs in this list
    lr_sched: update lr to value at matching epoch. Should be same length as epoch_sched
    bs_sched: update bs to value at matching epoch. Should be same length as epoch_sched
    """
    
    stats = list(stats_df.T.to_dict().values())
    print("[SETUP] Loaded {} epochs of stats".format(len(stats)))
    
    # preprocess data
    start = time()
    train_set = model.preprocess(train_set)
    test_set = model.preprocess(test_set)
    print("[SETUP] Pre-Processed in {} seconds".format(time() - start))
    
    history = []
    for epoch in range(min(n_epochs, len(stats))):
        
        start = time()
        
        # set stats
        stat = stats.pop(0)
        model.set_sim(stat)
        
        # run
        print("[Epoch {}]".format(epoch), end="")
        model.partial_fit(train_set)
        score = model.score(test_set)
        datum = {"epoch": epoch, "score": score, **model.get_params(), **model.meta_}
        print(" Score: {} in {} seconds".format(score, time() - start))
        history.append(datum)
        
        if epoch % log_interval == 0:
            write_stats(history, stats_dir, 'results-ep{}.csv'.format(epoch))

    return history

In [10]:
def run_sim(stats_df_loc, out_folder, timings, max_bs=99999999):
    
    print("Running simulation....\n\n")
    start = time()
    
    stats_df = pd.read_csv(stats_df_loc)
    
    if 'lr_' in stats_df.columns:
        stats_df = stats_df.rename(columns={"lr_": "partial_fit__lr", "damping_": "partial_fit__batch_size"})
    
    init_lr = stats_df['partial_fit__lr'][0]
    init_bs = stats_df['partial_fit__batch_size'][0]
    momentum = stats_df['optimizer__momentum'][0]
    device = "cpu" if not torch.cuda.is_available() else "cuda:0"

    args = dict(
        module=Wide_ResNet,
        module__depth=16,
        module__widen_factor=4,
        module__dropout_rate=0.3,
        module__num_classes=len(classes),
        loss=torch.nn.CrossEntropyLoss,
        optimizer=torch.optim.SGD,
        optimizer__lr=init_lr,
        optimizer__momentum=momentum,
        optimizer__nesterov=True,
        optimizer__weight_decay=0.5e-3,
        batch_size=init_bs,
        max_epochs=200,
        device=device,
        grads_per_worker=128,
        client=client,
        lr=init_lr,
        max_batch_size=max_bs
    )
    model = DaskClassifierSimulator(**args)
    model.set_times(timings['mult'], timings['score'], timings['deepcopy'], timings['grad128'])
    
    # train
    hist = train(model, 
             train_set, 
             test_set, 
             stats_df,
             n_epochs=200, 
             log_interval=20,
             stats_dir=out_folder
            )
    write_stats(hist, out_folder, 'results-final.csv')
    
    print('Finished simulation in {} seconds'.format(time() - start))
    

In [12]:
# test exp
timings = { 'mult': True, 'score': 0.0, 'deepcopy': 0.0, 'grad128': 0.0 }
stats_path = './exp-dask/stats/increasing-bs/exp-final.csv'
out_path = 'sim-results/test'
run_sim(stats_path, out_path, timings)

Running simulation....


[SETUP] Loaded 200 epochs of stats
[SETUP] Pre-Processed in 18.534586906433105 seconds
[Epoch 0] Score: 0.5943999886512756 in 10.096720933914185 seconds
[Epoch 1] Score: 0.6757999658584595 in 7.93016505241394 seconds
[Epoch 2] Score: 0.7128999829292297 in 6.990623235702515 seconds
Finished simulation in 43.66578984260559 seconds


In [11]:
## TIMINGS FOR NORMAL EXPERIMENTS
SCORE_TIME = 0.0 # 5.39407711148262
DEEPCOPY_TIME = 0.05855  # seconds
GRAD_TIME_128 = 0.07832  # seconds
timings = { 'mult': False, 'score': SCORE_TIME, 'deepcopy': DEEPCOPY_TIME, 'grad128': GRAD_TIME_128 }

In [18]:
# exp 1 - dec-lr-1-machine 
stats_path = './exp-dask/stats/decreasing-lr/exp--final.csv'
out_path = 'sim-results/dec-lr-1-machine'
run_sim(stats_path, out_path, timings)

Running simulation....


[SETUP] Loaded 200 epochs of stats
[SETUP] Pre-Processed in 17.500604391098022 seconds
[Epoch 0] Score: 0.5990999937057495 in 60.29955196380615 seconds
[Epoch 1] Score: 0.6762999892234802 in 60.30539417266846 seconds
[Epoch 2] Score: 0.7156999707221985 in 59.774914503097534 seconds
[Epoch 3] Score: 0.7777999639511108 in 60.207035779953 seconds
[Epoch 4] Score: 0.7678999900817871 in 59.78869414329529 seconds
[Epoch 5] Score: 0.7994999885559082 in 60.308109283447266 seconds
[Epoch 6] Score: 0.8030999898910522 in 59.963244676589966 seconds
[Epoch 7] Score: 0.8100000023841858 in 60.10037112236023 seconds
[Epoch 8] Score: 0.8186999559402466 in 59.96827411651611 seconds
[Epoch 9] Score: 0.8077999949455261 in 60.28385829925537 seconds
[Epoch 10] Score: 0.8211999535560608 in 60.00210118293762 seconds
[Epoch 11] Score: 0.8321999907493591 in 60.082077980041504 seconds
[Epoch 12] Score: 0.8306999802589417 in 59.640068769454956 seconds
[Epoch 13] Score: 0.8325999975204468 

In [25]:
# exp 5 - large bs 1 1 machine
stats_path = './exp-dask/stats/large-bs-0/exp-final.csv'
out_path = 'sim-results/large-bs1-1-machine'
run_sim(stats_path, out_path, timings, max_bs=5120)

Running simulation....


[SETUP] Loaded 200 epochs of stats
[SETUP] Pre-Processed in 17.082221746444702 seconds
[Epoch 0] Score: 0.5884999632835388 in 30.94543957710266 seconds
[Epoch 1] Score: 0.6516000032424927 in 30.91340970993042 seconds
[Epoch 2] Score: 0.7285000085830688 in 30.86415672302246 seconds
[Epoch 3] Score: 0.7662000060081482 in 31.18868088722229 seconds
[Epoch 4] Score: 0.7896999716758728 in 30.9765408039093 seconds
[Epoch 5] Score: 0.7972999811172485 in 30.914204597473145 seconds
[Epoch 6] Score: 0.8024999499320984 in 31.085127592086792 seconds
[Epoch 7] Score: 0.8089999556541443 in 30.978079080581665 seconds
[Epoch 8] Score: 0.8148999810218811 in 30.929124355316162 seconds
[Epoch 9] Score: 0.8120999932289124 in 30.941798210144043 seconds
[Epoch 10] Score: 0.8294000029563904 in 30.995038747787476 seconds
[Epoch 11] Score: 0.8185999989509583 in 30.99557065963745 seconds
[Epoch 12] Score: 0.8331999778747559 in 30.947920083999634 seconds
[Epoch 13] Score: 0.84169995784759

In [12]:
# exp 2 - inc-bs
stats_path = './exp-dask/stats/increasing-bs/exp-final.csv'
out_path = 'sim-results/inc-bs-1-machine'
run_sim(stats_path, out_path, timings)

Running simulation....


[SETUP] Loaded 200 epochs of stats
[SETUP] Pre-Processed in 18.031914472579956 seconds
[Epoch 0] Score: 0.5943999886512756 in 67.52644038200378 seconds
[Epoch 1] Score: 0.6757999658584595 in 59.98326015472412 seconds
[Epoch 2] Score: 0.7128999829292297 in 59.854674100875854 seconds
[Epoch 3] Score: 0.7423999905586243 in 59.890366554260254 seconds
[Epoch 4] Score: 0.7707999944686891 in 60.007487058639526 seconds
[Epoch 5] Score: 0.7875999808311462 in 60.24218225479126 seconds
[Epoch 6] Score: 0.7892999649047852 in 59.979164361953735 seconds
[Epoch 7] Score: 0.8071999549865723 in 60.02729058265686 seconds
[Epoch 8] Score: 0.8120999932289124 in 59.98218131065369 seconds
[Epoch 9] Score: 0.8226000070571899 in 59.84120202064514 seconds
[Epoch 10] Score: 0.8353999853134155 in 59.62069082260132 seconds
[Epoch 11] Score: 0.8224999904632568 in 59.88772654533386 seconds
[Epoch 12] Score: 0.8246999979019165 in 60.31302237510681 seconds
[Epoch 13] Score: 0.8355000019073486

In [13]:
# exp 3 - hybrid 1 1 machine
stats_path = './exp-dask/stats/hybrid/exp-final.csv'
out_path = 'sim-results/hybrid1-1-machine'
run_sim(stats_path, out_path, timings)

Running simulation....


[SETUP] Loaded 200 epochs of stats
[SETUP] Pre-Processed in 18.295875072479248 seconds
[Epoch 0] Score: 0.6014999747276306 in 60.282591104507446 seconds
[Epoch 1] Score: 0.6804999709129333 in 59.86171579360962 seconds
[Epoch 2] Score: 0.7328000068664551 in 60.11501145362854 seconds
[Epoch 3] Score: 0.7621999979019165 in 59.94831562042236 seconds
[Epoch 4] Score: 0.7834999561309814 in 60.12259292602539 seconds
[Epoch 5] Score: 0.8009999990463257 in 59.74931573867798 seconds
[Epoch 6] Score: 0.8064000010490417 in 60.086275577545166 seconds
[Epoch 7] Score: 0.8100000023841858 in 60.28357291221619 seconds
[Epoch 8] Score: 0.8197999596595764 in 59.819730043411255 seconds
[Epoch 9] Score: 0.812999963760376 in 59.997806549072266 seconds
[Epoch 10] Score: 0.8154999613761902 in 59.752153158187866 seconds
[Epoch 11] Score: 0.8373000025749207 in 59.817837953567505 seconds
[Epoch 12] Score: 0.8402000069618225 in 60.43029856681824 seconds
[Epoch 13] Score: 0.831900000572204

In [14]:
# exp 4 - hybrid 2 1 machine
stats_path = './exp-dask/stats/hybrid-2/exp-final.csv'
out_path = 'sim-results/hybrid2-1-machine'
run_sim(stats_path, out_path, timings)

Running simulation....


[SETUP] Loaded 200 epochs of stats
[SETUP] Pre-Processed in 20.62927269935608 seconds
[Epoch 0] Score: 0.6290000081062317 in 59.91703271865845 seconds
[Epoch 1] Score: 0.694599986076355 in 60.07186412811279 seconds
[Epoch 2] Score: 0.7508999705314636 in 60.29324007034302 seconds
[Epoch 3] Score: 0.7793999910354614 in 59.68764781951904 seconds
[Epoch 4] Score: 0.8016999959945679 in 59.633214712142944 seconds
[Epoch 5] Score: 0.8024999499320984 in 60.19446587562561 seconds
[Epoch 6] Score: 0.8096999526023865 in 60.63145613670349 seconds
[Epoch 7] Score: 0.8320999741554259 in 60.047417879104614 seconds
[Epoch 8] Score: 0.8307999968528748 in 59.9759304523468 seconds
[Epoch 9] Score: 0.8369999527931213 in 60.12659549713135 seconds
[Epoch 10] Score: 0.8292999863624573 in 59.90186953544617 seconds
[Epoch 11] Score: 0.8430999517440796 in 59.623199224472046 seconds
[Epoch 12] Score: 0.8443999886512756 in 59.77467894554138 seconds
[Epoch 13] Score: 0.8333999514579773 in 

In [15]:
# exp 6 - large bs 2 1 machine
stats_path = './exp-dask/stats/large-bs-1/exp-final.csv'
out_path = 'sim-results/large-bs2-1-machine'
run_sim(stats_path, out_path, timings, max_bs=5120)

Running simulation....


[SETUP] Loaded 200 epochs of stats
[SETUP] Pre-Processed in 18.92701292037964 seconds
[Epoch 0] Score: 0.4782999753952026 in 31.559585571289062 seconds
[Epoch 1] Score: 0.5977999567985535 in 31.12877869606018 seconds
[Epoch 2] Score: 0.665399968624115 in 31.154720783233643 seconds
[Epoch 3] Score: 0.6798999905586243 in 31.17481565475464 seconds
[Epoch 4] Score: 0.6807999610900879 in 31.187304496765137 seconds
[Epoch 5] Score: 0.7182999849319458 in 31.188523292541504 seconds
[Epoch 6] Score: 0.7193999886512756 in 31.131200790405273 seconds
[Epoch 7] Score: 0.7482999563217163 in 31.13277268409729 seconds
[Epoch 8] Score: 0.6961999535560608 in 31.131117343902588 seconds
[Epoch 9] Score: 0.7603999972343445 in 31.12556290626526 seconds
[Epoch 10] Score: 0.7512999773025513 in 31.473087549209595 seconds
[Epoch 11] Score: 0.7337999939918518 in 31.136924982070923 seconds
[Epoch 12] Score: 0.7601000070571899 in 31.121317863464355 seconds
[Epoch 13] Score: 0.7540000081062

In [16]:
# timings for multi machine set ups
SCORE_TIME = 0.0 # 5.39407711148262
DEEPCOPY_TIME = 0.05855  # seconds
GRAD_TIME_128 = 0.07832  # seconds
timings = { 'mult': True, 'score': SCORE_TIME, 'deepcopy': DEEPCOPY_TIME, 'grad128': GRAD_TIME_128 }

In [17]:
# exp 7 - dec-lr-P-machine 
stats_path = './exp-dask/stats/decreasing-lr/exp--final.csv'
out_path = 'sim-results/dec-lr-mult-machines'
run_sim(stats_path, out_path, timings)

Running simulation....


[SETUP] Loaded 200 epochs of stats
[SETUP] Pre-Processed in 19.3242449760437 seconds
[Epoch 0] Score: 0.5990999937057495 in 60.27368378639221 seconds
[Epoch 1] Score: 0.6762999892234802 in 60.63890600204468 seconds
[Epoch 2] Score: 0.7156999707221985 in 60.39743447303772 seconds
[Epoch 3] Score: 0.7777999639511108 in 59.95809054374695 seconds
[Epoch 4] Score: 0.7678999900817871 in 60.05986166000366 seconds
[Epoch 5] Score: 0.7994999885559082 in 59.853527307510376 seconds
[Epoch 6] Score: 0.8030999898910522 in 60.19325876235962 seconds
[Epoch 7] Score: 0.8100000023841858 in 59.97223877906799 seconds
[Epoch 8] Score: 0.8186999559402466 in 60.05410051345825 seconds
[Epoch 9] Score: 0.8077999949455261 in 60.01777100563049 seconds
[Epoch 10] Score: 0.8211999535560608 in 59.60686278343201 seconds
[Epoch 11] Score: 0.8321999907493591 in 59.68800902366638 seconds
[Epoch 12] Score: 0.8306999802589417 in 59.800074100494385 seconds
[Epoch 13] Score: 0.8325999975204468 in 

In [18]:
# exp 8 - increasing batch p machines
stats_path = './exp-dask/stats/increasing-bs/exp-final.csv'
out_path = 'sim-results/inc-bs-mult-machines'
run_sim(stats_path, out_path, timings)

Running simulation....


[SETUP] Loaded 200 epochs of stats
[SETUP] Pre-Processed in 18.085535049438477 seconds
[Epoch 0] Score: 0.5943999886512756 in 59.921546936035156 seconds
[Epoch 1] Score: 0.6757999658584595 in 59.84012508392334 seconds
[Epoch 2] Score: 0.7128999829292297 in 59.92027521133423 seconds
[Epoch 3] Score: 0.7423999905586243 in 59.985854387283325 seconds
[Epoch 4] Score: 0.7707999944686891 in 60.23252463340759 seconds
[Epoch 5] Score: 0.7875999808311462 in 60.11578917503357 seconds
[Epoch 6] Score: 0.7892999649047852 in 60.01540780067444 seconds
[Epoch 7] Score: 0.8071999549865723 in 60.27407908439636 seconds
[Epoch 8] Score: 0.8120999932289124 in 60.134197473526 seconds
[Epoch 9] Score: 0.8226000070571899 in 60.25694417953491 seconds
[Epoch 10] Score: 0.8353999853134155 in 59.67681837081909 seconds
[Epoch 11] Score: 0.8224999904632568 in 59.833309173583984 seconds
[Epoch 12] Score: 0.8246999979019165 in 60.0260808467865 seconds
[Epoch 13] Score: 0.8355000019073486 in 

In [19]:
# exp 9 - hybrid 1 P machine
stats_path = './exp-dask/stats/hybrid/exp-final.csv'
out_path = 'sim-results/hybrid1-mult-machines'
run_sim(stats_path, out_path, timings)

Running simulation....


[SETUP] Loaded 200 epochs of stats
[SETUP] Pre-Processed in 18.729305028915405 seconds
[Epoch 0] Score: 0.6014999747276306 in 59.66806960105896 seconds
[Epoch 1] Score: 0.6804999709129333 in 59.92812371253967 seconds
[Epoch 2] Score: 0.7328000068664551 in 59.7060968875885 seconds
[Epoch 3] Score: 0.7621999979019165 in 60.385709047317505 seconds
[Epoch 4] Score: 0.7834999561309814 in 60.14630341529846 seconds
[Epoch 5] Score: 0.8009999990463257 in 60.19202995300293 seconds
[Epoch 6] Score: 0.8064000010490417 in 59.82971477508545 seconds
[Epoch 7] Score: 0.8100000023841858 in 60.022382497787476 seconds
[Epoch 8] Score: 0.8197999596595764 in 59.40756368637085 seconds
[Epoch 9] Score: 0.812999963760376 in 59.96042585372925 seconds
[Epoch 10] Score: 0.8154999613761902 in 60.54906606674194 seconds
[Epoch 11] Score: 0.8373000025749207 in 59.802112102508545 seconds
[Epoch 12] Score: 0.8402000069618225 in 59.78507709503174 seconds
[Epoch 13] Score: 0.8319000005722046 in

In [20]:
# exp 10 - hybrid 2 P machine
stats_path = './exp-dask/stats/hybrid-2/exp-final.csv'
out_path = 'sim-results/hybrid2-mult-machines'
run_sim(stats_path, out_path, timings)

Running simulation....


[SETUP] Loaded 200 epochs of stats
[SETUP] Pre-Processed in 20.09123206138611 seconds
[Epoch 0] Score: 0.6290000081062317 in 60.102150201797485 seconds
[Epoch 1] Score: 0.694599986076355 in 60.04787731170654 seconds
[Epoch 2] Score: 0.7508999705314636 in 60.01671075820923 seconds
[Epoch 3] Score: 0.7793999910354614 in 59.664647579193115 seconds
[Epoch 4] Score: 0.8016999959945679 in 60.037010192871094 seconds
[Epoch 5] Score: 0.8024999499320984 in 59.86994409561157 seconds
[Epoch 6] Score: 0.8096999526023865 in 60.1696035861969 seconds
[Epoch 7] Score: 0.8320999741554259 in 60.16039276123047 seconds
[Epoch 8] Score: 0.8307999968528748 in 59.62686729431152 seconds
[Epoch 9] Score: 0.8369999527931213 in 60.179363489151 seconds
[Epoch 10] Score: 0.8292999863624573 in 59.789252519607544 seconds
[Epoch 11] Score: 0.8430999517440796 in 60.2604866027832 seconds
[Epoch 12] Score: 0.8443999886512756 in 59.8801474571228 seconds
[Epoch 13] Score: 0.8333999514579773 in 60.

In [21]:
# exp 11 - large bs 1 P machine
stats_path = './exp-dask/stats/large-bs-0/exp-final.csv'
out_path = 'sim-results/large-bs1-mult-machines'
run_sim(stats_path, out_path, timings, max_bs=5120)

Running simulation....


[SETUP] Loaded 200 epochs of stats
[SETUP] Pre-Processed in 19.54126262664795 seconds
[Epoch 0] Score: 0.5884999632835388 in 31.073299646377563 seconds
[Epoch 1] Score: 0.6516000032424927 in 30.98455262184143 seconds
[Epoch 2] Score: 0.7285000085830688 in 31.271871089935303 seconds
[Epoch 3] Score: 0.7662000060081482 in 30.939972162246704 seconds
[Epoch 4] Score: 0.7896999716758728 in 30.96862769126892 seconds
[Epoch 5] Score: 0.7972999811172485 in 30.894929885864258 seconds
[Epoch 6] Score: 0.8024999499320984 in 30.95479130744934 seconds
[Epoch 7] Score: 0.8089999556541443 in 30.865525245666504 seconds
[Epoch 8] Score: 0.8148999810218811 in 30.911786317825317 seconds
[Epoch 9] Score: 0.8120999932289124 in 30.891368627548218 seconds
[Epoch 10] Score: 0.8294000029563904 in 30.96508026123047 seconds
[Epoch 11] Score: 0.8185999989509583 in 30.946865797042847 seconds
[Epoch 12] Score: 0.8331999778747559 in 31.319865226745605 seconds
[Epoch 13] Score: 0.841699957847

In [22]:
# exp 12 - large bs 2 P machine
stats_path = './exp-dask/stats/large-bs-1/exp-final.csv'
out_path = 'sim-results/large-bs2-mult-machines'
run_sim(stats_path, out_path, timings, max_bs=5120)

Running simulation....


[SETUP] Loaded 200 epochs of stats
[SETUP] Pre-Processed in 18.18881916999817 seconds
[Epoch 0] Score: 0.4782999753952026 in 31.06590509414673 seconds
[Epoch 1] Score: 0.5977999567985535 in 30.964808225631714 seconds
[Epoch 2] Score: 0.665399968624115 in 31.301671981811523 seconds
[Epoch 3] Score: 0.6798999905586243 in 30.854966640472412 seconds
[Epoch 4] Score: 0.6807999610900879 in 30.84601926803589 seconds
[Epoch 5] Score: 0.7182999849319458 in 30.89863157272339 seconds
[Epoch 6] Score: 0.7193999886512756 in 30.911365032196045 seconds
[Epoch 7] Score: 0.7482999563217163 in 30.863516330718994 seconds
[Epoch 8] Score: 0.6961999535560608 in 30.917965173721313 seconds
[Epoch 9] Score: 0.7603999972343445 in 30.90145778656006 seconds
[Epoch 10] Score: 0.7512999773025513 in 30.88135051727295 seconds
[Epoch 11] Score: 0.7337999939918518 in 30.926563262939453 seconds
[Epoch 12] Score: 0.7601000070571899 in 31.213849782943726 seconds
[Epoch 13] Score: 0.75400000810623